from database.DataBaseModels import *

import networkx as nx
import json

In [2]:
author_graph = nx.MultiDiGraph()

In [3]:
for citation in db.Citation.select():
    for author_to in citation.article_id.article__author_article.author_id:
        for author_from in citation.cited_by.article__author_article.author_id:
            author_graph.add_edge(author_from, author_to)

In [4]:
print(len(author_graph.edges))
print(len(author_graph.nodes))

104307
5791


In [5]:
# Removing selfloops edges
author_graph.remove_edges_from([edge for edge in author_graph.selfloop_edges()])

In [22]:
# List of authors pairs who cited each other
result = []

for node in author_graph.nodes:
    for successor in author_graph.successors(node):
        if node in author_graph.successors(successor):
            result.append([successor, node])


for sublist in result:
    result.remove([sublist[1], sublist[0]])
    
result_dict = {}

for pair in enumerate(result):
    result_dict[pair[0]] = {pair[1][0].name: pair[1][0].author_id,
                           pair[1][1].name: pair[1][1].author_id}

In [25]:
for key in result_dict.keys():
    print(result_dict[key])

{'Charles Consel': 9790, 'Dawson R. Engler': 10130}
{'Nico Zazworka': 3656, 'Yuepu Guo': 3648}
{'Yuepu Guo': 3648, 'Philippe Kruchten': 3650}
{'Alan MacCormack': 3652, 'Yuepu Guo': 3648}
{'Carolyn Seaman': 3655, 'Yuepu Guo': 3648}
{'Yuanfang Cai': 3257, 'Yuepu Guo': 3648}
{'Rick Kazman': 3649, 'Yuepu Guo': 3648}
{'Yuepu Guo': 3648, 'Ipek Ozkaya': 2439}
{'Robert Nord': 3653, 'Yuepu Guo': 3648}
{'Yuepu Guo': 3648, 'Nanette Brown': 3647}
{'Christoffer Quist Adamsen': 6671, 'Frank Tip': 6672}
{'Frank Tip': 6672, 'Magnus Madsen': 2809}
{'Frank Tip': 6672, 'Anders Møller': 2812}
{'Frank Tip': 6672, 'Karthik Pattabiraman': 7775}
{'Ali Mesbah': 2470, 'Frank Tip': 6672}
{'Chao Wang': 2145, 'Frank Tip': 6672}
{'Tommi Mikkonen': 2685, 'Arto Salminen': 8370}
{'Reza Matinnejad': 2361, 'Shiva Nejati': 2362}
{'Reza Matinnejad': 2361, 'Lionel C. Briand': 2363}
{'Yuanfang Cai': 3257, 'Sunny Wong': 8188}
{'Rick Kazman': 3649, 'Sunny Wong': 8188}
{'Igor Steinmacher': 3971, 'Tayana Conte': 8533}
{'Arie va

{'Christian Bird': 2435, 'Emerson Murphy-Hill': 2481}
{'Emerson Murphy-Hill': 2481, 'Will Snipes': 5062}
{'Emerson Murphy-Hill': 2481, 'Nachiappan Nagappan': 2529}
{'Emerson Murphy-Hill': 2481, 'Meiyappan Nagappan': 2523}
{'Emerson Murphy-Hill': 2481, 'Sarah Elder': 4840}
{'Emerson Murphy-Hill': 2481, 'Thomas Zimmermann': 2488}
{'Chong Tang': 6591, 'Hamid Bagheri': 2002}
{'Laura Inozemtseva': 2298, 'Yue Jia': 1961}
{'Laura Inozemtseva': 2298, 'Annibale Panichella': 2458}
{'Gordon Fraser': 2447, 'Laura Inozemtseva': 2298}
{'Laura Inozemtseva': 2298, 'Mauro Pezzè': 2265}
{'Mark Harman': 1960, 'Laura Inozemtseva': 2298}
{'Antti Nieminen': 8086, 'Janne Lautamäki': 8085}
{'Terhi Kilamo': 5861, 'Antti Nieminen': 8086}
{'Antti Nieminen': 8086, 'Jarmo Palviainen': 8100}
{'Martin C. Carlisle': 10160, 'Anne Rogers': 10159}
{'Yingfei Xiong': 2609, 'Claire Le Goues': 2231}
{'David Lo': 2230, 'Claire Le Goues': 2231}
{'Claire Le Goues': 2231, 'Thomas Durieux': 6798}
{'Claire Le Goues': 2231, 'Runfa

{'Brad Vander Zanden': 9638, 'Scott E. Hudson': 9407}
{'Roger King': 9408, 'Scott E. Hudson': 9407}
{'David S. Kosbie': 9635, 'Scott E. Hudson': 9407}
{'Dario A. Giuse': 9633, 'Scott E. Hudson': 9407}
{'Brad A. Myers': 9602, 'Scott E. Hudson': 9407}
{'Pedro Szekely': 9455, 'Scott E. Hudson': 9407}
{'Ciro Baron Neto': 8932, 'Sebastian Hellmann': 4748}
{'Dimitris Kontokostas': 4747, 'Ciro Baron Neto': 8932}
{'Michael J. Carey': 8640, 'Scott L. Vandenberg': 9314}
{'Jan Oliver Ringert': 2153, 'Bernhard Rumpe': 2804}
{'Nico Zazworka': 3656, 'Forrest Shull': 8195}
{'Enrico Budianto': 2490, 'Atul Sadhu': 2493}
{'Shweta Shinde': 2491, 'Enrico Budianto': 2490}
{'Enrico Budianto': 2490, 'Prateek Saxena': 2494}
{'Enrico Budianto': 2490, 'Hung Dang': 2492}
{'Dengfeng Li': 2312, 'Xia Zeng': 5010}
{'Franceska Xhakaj': 7791, 'Anders Møller': 2812}
{'Nicolas Sannier': 2656, 'Jean-Marc Davril': 2657}
{'Mathieu Acher': 2625, 'Nicolas Sannier': 2656}
{'Brandon Lucia': 6495, 'Malavika Samak': 2453}
{'Mark

In [26]:
both_side_citations_graph = nx.Graph()

both_side_citations_graph.add_edges_from([[author[0].name, author[1].name] for author in result])

In [27]:
len(result)

4861